<a href="https://colab.research.google.com/github/mparvezrashid/ECE542-project-F/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ktrain
from ktrain import text
import numpy as np
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from bs4 import BeautifulSoup
import string
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

C:\Users\André Motta\AppData\Roaming\Python\Python37\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('C3_anonymized.csv', delimiter=',', usecols=['comment_text','constructive_binary'])

In [3]:
X, X_test, y, y_test = train_test_split(df['comment_text'], df['constructive_binary'],stratify=df['constructive_binary'],test_size=0.15)
X_train, X_val, y_train, y_val = train_test_split(X, y,stratify=y,test_size=0.15)

y_train = LabelEncoder().fit_transform(y_train)
y_test = LabelEncoder().fit_transform(y_test)
y_val = LabelEncoder().fit_transform(y_val)

In [4]:
X_train = np.asarray(X_train)
X_val = np.asarray(X_val)
X_test = np.asarray(X_test)

In [5]:
transform = text.Transformer('bert-base-uncased', maxlen=150, class_names=[0,1])
train_final = transform.preprocess_train(X_train, y_train)
val_final = transform.preprocess_test(X_val, y_val)
model = transform.get_classifier()
lrnr = ktrain.get_learner(model, train_data=train_final, val_data=val_final, batch_size=32)

preprocessing train...
language: en
train sequence lengths:
	mean : 72
	95percentile : 216
	99percentile : 370


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 69
	95percentile : 199
	99percentile : 344


In [6]:
lrnr.fit_onecycle(1e-3, 4)



begin training using onecycle policy with max lr of 0.001...
Epoch 1/4


ResourceExhaustedError:  OOM when allocating tensor with shape[32,12,150,150] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node tf_bert_for_sequence_classification/bert/encoder/layer_._10/attention/self/einsum/Einsum (defined at C:\Users\André Motta\AppData\Roaming\Python\Python37\site-packages\transformers\models\bert\modeling_tf_bert.py:358) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_19826]

Errors may have originated from an input operation.
Input Source operations connected to node tf_bert_for_sequence_classification/bert/encoder/layer_._10/attention/self/einsum/Einsum:
 tf_bert_for_sequence_classification/bert/encoder/layer_._10/attention/self/Mul (defined at C:\Users\André Motta\AppData\Roaming\Python\Python37\site-packages\transformers\models\bert\modeling_tf_bert.py:357)	
 tf_bert_for_sequen

In [ ]:
prdtct = ktrain.get_predictor(lrnr.model, preproc=transform)
y_pred = prdct.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))
print(classification_report(y_test, y_pred, target_names=['Not constructive', 'Constructive']))
